In [21]:
from fashionData import *
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [22]:
train = FashionData('data/fashion-mnist_train.csv')

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)



In [24]:
network = Net()
print(network)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [25]:
learning_rate = 0.01
optimizer = optim.SGD(network.parameters(), lr=learning_rate)
loss_criterion = nn.CrossEntropyLoss()

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs=15
batch_size = 100


train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)

n_total_steps = len(train_loader)

for epoch in range(n_epochs):
    for i, (labels, images) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass and loss calculation
        outputs = network(images)
        loss = loss_criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{n_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

/var/folders/2y/bnxt2f3n7z5btz_1gx1hdklh0000gn/T/ipykernel_8041/1954227324.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Epoch [1/15], Step [100/600], Loss: 1.3493
Epoch [1/15], Step [200/600], Loss: 0.9450
Epoch [1/15], Step [300/600], Loss: 0.9920
Epoch [1/15], Step [400/600], Loss: 1.0244
Epoch [1/15], Step [500/600], Loss: 0.9960
Epoch [1/15], Step [600/600], Loss: 0.9683
Epoch [2/15], Step [100/600], Loss: 0.9346
Epoch [2/15], Step [200/600], Loss: 0.8453
Epoch [2/15], Step [300/600], Loss: 0.5957
Epoch [2/15], Step [400/600], Loss: 1.0050
Epoch [2/15], Step [500/600], Loss: 0.7829
Epoch [2/15], Step [600/600], Loss: 0.7071
Epoch [3/15], Step [100/600], Loss: 0.7644
Epoch [3/15], Step [200/600], Loss: 0.6007
Epoch [3/15], Step [300/600], Loss: 0.6361
Epoch [3/15], Step [400/600], Loss: 0.5340
Epoch [3/15], Step [500/600], Loss: 0.5563
Epoch [3/15], Step [600/600], Loss: 0.5495
Epoch [4/15], Step [100/600], Loss: 0.9556
Epoch [4/15], Step [200/600], Loss: 0.8029
Epoch [4/15], Step [300/600], Loss: 0.6622
Epoch [4/15], Step [400/600], Loss: 0.6566
Epoch [4/15], Step [500/600], Loss: 0.6502
Epoch [4/15

In [27]:
test = FashionData('data/fashion-mnist_test.csv')
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=True)


with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)
    for labels, images in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = network(images)

        # max returns (output_value ,index)
        _, predicted = torch.max(outputs, 1)
        n_correct += (predicted == labels).sum().item()

    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')

/var/folders/2y/bnxt2f3n7z5btz_1gx1hdklh0000gn/T/ipykernel_8041/1954227324.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Accuracy of the network on the 10000 test images: 82.01 %
